# Phase 4: Generative AI

### In this phase, we integrate Generative AI into our Travel Recommendadtion System. This AI integration will improve the Travel Recommendadtion System by providing detailed information about destinations based on user input. To achieve this, we will use one of the LAMA Generative AI model though Groq framework.

## Load libraries

In [1]:
import pandas as pd
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")

## Load datasets and models

In [2]:
# load cleaned and clustered datasets
cleaned_df = pd.read_csv('data/cleaned_df.csv')
clustered_dataset = pd.read_csv('data/clustered_dataset.csv')

# Load destination normalization model
to_model = pickle.load(open('models/to_model.sav', 'rb')) 

# load Random Forest model
RF_Model = pickle.load(open('models/RF_Model.sav', 'rb')) 

# load K-Means model
kmeans_model = pickle.load(open('models/kmeans_model.sav', 'rb')) 

## Example input data

In [5]:
# Define features
features = ['from','price_x', 'time','flightType', 'agency', 'days', 'price_y', 'age', 'gender','company']

# features
X = cleaned_df[features]

# input data
input_data = X.iloc[3,:]

## Predict the destination using Random Forest Model

In [6]:
predicted_destination = RF_Model.predict(np.array(input_data).reshape(1, -1))
predicted_destination_label = to_model.inverse_transform(predicted_destination)
print('Predicted Destination: ', predicted_destination_label[0])

Predicted Destination:  Aracaju (SE)


In [7]:
# find cluster of the current input data
predicted_cluster = kmeans_model.predict(np.array(input_data).reshape(1, -1))
pred_cluster = predicted_cluster[0]

# Make Recommendations

In [8]:
# Fetch similar users in the same cluster
similar_users = clustered_dataset[clustered_dataset['cluster_kmeans'] == pred_cluster]
top_destinations = similar_users['to'].value_counts().head(2).index.tolist()
top_hotels = similar_users['name_x'].value_counts().head(3).index.tolist()

In [9]:
# add the predicted destination to the top destinations 
if predicted_destination_label[0] not in top_destinations:
    top_destinations.insert(0, predicted_destination_label[0])

In [10]:
recommendation = f"""
Based on your input, we recommend the following:
- Top Destinations: {', '.join(top_destinations)}
- Recommended Hotels: {', '.join(top_hotels)}
- Average Flight Price: ${similar_users['price_x'].mean():.2f}
- Average Stay Cost: ${similar_users['total'].mean():.2f}
"""

In [11]:
print(recommendation)


Based on your input, we recommend the following:
- Top Destinations: Aracaju (SE), Florianopolis (SC), Campo Grande (MS)
- Recommended Hotels: Hotel K, Hotel AF, Hotel CB
- Average Flight Price: $1009.21
- Average Stay Cost: $510.16



# AI Integration

## Load libraries

In [12]:
import os
from groq import Groq

## API Key

In [13]:
os.environ["GROQ_API_KEY"] = "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"

## Function for generating recommendations

In [14]:
def generate_recommendation(cluster, similar_users,top_destinations,top_hotels,template_type="brief"):  
    # Define prompt templates
    brief_prompt = f"""
    Based on your travel history and spending habits, we recommend the following:
    - Top Destinations: {', '.join(top_destinations)}
    - Recommended Hotels: {', '.join(top_hotels)}
    - Average Flight Price: ${similar_users['price_x'].mean():.2f}
    - Average Stay Cost: ${similar_users['total'].mean():.2f}
    """
    
    detailed_prompt = f"""
    You are an AI travel assistant. A corporate traveler has a history of business trips and prefers optimized recommendations.
    The user is part of cluster {cluster}, which includes travelers with similar spending patterns.
    
    Suggest the best travel options based on:
    - Preferred destinations: {', '.join(top_destinations)}
    - Recommended hotels: {', '.join(top_hotels)}
    - Estimated budget: Flight (${similar_users['price_x'].mean():.2f}), Hotel (${similar_users['total'].mean():.2f})
    
    Provide a detailed explanation of why these choices suit the user’s profile.
    """
    
    # Select the prompt type
    prompt = brief_prompt if template_type == "brief" else detailed_prompt
    
    
    # Create the Groq client
    client = Groq(api_key=os.environ.get("GROQ_API_KEY"), )

    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "system", "content": "You are a travel recommendation assistant."},
                  {"role": "user", "content": prompt}],
        temperature=1,
        max_tokens=1024
    )
           
    return response.choices[0].message.content

### Brief Recommendation

In [15]:
generated_recommendation_brief = generate_recommendation(pred_cluster, similar_users, top_destinations, top_hotels, template_type="brief")

print("--- Brief Recommendation ---")
print(generated_recommendation_brief)


--- Brief Recommendation ---
It seems like I have some interesting travel proposals based on my past preferences and spending habits.

To recap, here are the top destination recommendations:

1. **Aracaju (SE)**: I've never visited Aracaju before, but I'm curious about the enchanting coastal city in the state of Sergipe. After traveling through Brazil, I'm excited to explore this lesser-known gem.

2. **Florianopolis (SC)**: As a fan of stunning beaches and picturesque landscapes, Florianopolis is a perfect fit. I love the idea of relaxing in this beautiful city on the southeastern coast of Brazil.

3. **Campo Grande (MS)**: This option is a bit more unusual, but I'm intrigued by the rich culture and natural beauty of Mato Grosso do Sul. It's time to explore the Pantanal region and experience the Wild West of Brazil.

Based on my past hotel preferences, I'm considering the following options:

1. **Hotel K**: This reliable hotel chain tends to offer excellent service and comfortable acc

### Detailed Recommendation

In [16]:
generated_recommendation_detailed = generate_recommendation(pred_cluster, similar_users, top_destinations, top_hotels, template_type="detailed")
print("\n--- Detailed Recommendation ---")
print(generated_recommendation_detailed)



--- Detailed Recommendation ---
**Personalized Travel Recommendations for Cluster 1 Corporate Traveler**

Based on the user's profile, I've analyzed their preferences and budget to provide optimized travel suggestions for their next business trip.

**Destination Recommendations**

Considering the user's fondness for visiting Aracaju (SE), Florianopolis (SC), and Campo Grande (MS), I've ranked the destinations based on factors such as accessibility, business relevance, and personal interest.

1. **Top Recommendation: Florianopolis (SC)**
Florianopolis is a strategic destination for business travelers, offering a mix of relaxation and productivity. Its beaches, cultural attractions, and modern infrastructure make it an ideal location for conferences and corporate events. Flight duration from major cities is approximately 2 hours, making it easily accessible.
2. **Secondary Recommendation: Aracaju (SE)**
Aracaju offers a unique blend of urban and natural attractions, with its beautiful b

---------------------------------------------------------------------       

# Demonstration of the Templates and the Differences between their Outcomes

## Differences Between the Brief and Detailed Templates
### The Brief Template provides a concise and straightforward overview of travel recommendations, listing top destinations, recommended hotels, and average costs in a simple bullet-point format. It focuses on delivering key information quickly without delving into detailed explanations or personalization. In contrast, the Detailed Template offers a comprehensive and structured analysis, including a rationale for the recommendations, a deeper explanation of how the choices align with the user’s profile, and even suggests modifications to the itinerary (e.g., swapping Aracaju for Vitoria). It also breaks down the estimated budget and explains how it fits the user’s spending patterns, making it more personalized and context-rich. While the Brief Template is easy to read and quick to digest, the Detailed Template is more informative, adaptive, and tailored to the user’s specific needs.

## Outcomes of Each Template
### The Brief Template produces a quick and easy-to-read summary of travel recommendations, ideal for users who want a fast overview without needing in-depth explanations or personalized insights. It highlights the top destinations, hotels, and costs in a clear and concise manner. On the other hand, the Detailed Template generates a thorough and personalized travel plan that considers the user’s preferences, spending habits, and cluster profile. It provides a detailed explanation of why the recommendations are suitable, suggests alternative options, and offers a breakdown of the budget, making it more informative and tailored. The Brief Template is perfect for users seeking a snapshot of their options, while the Detailed Template is better suited for those who want a well-reasoned and customized travel plan.

## Best Template:
### The choice between the two templates depends on the user’s needs and preferences. The Brief Template is better for users who prefer quick, concise information and do not require detailed explanations or personalized insights. It is ideal for those looking for a fast overview of travel options. In contrast, the Detailed Template is better for users who want a comprehensive, personalized travel plan with detailed reasoning behind the recommendations. It is particularly valuable for users who value tailored suggestions and are willing to spend more time reviewing a more complex but highly relevant travel plan. In summary, the Brief Template is suitable for simplicity and speed, while the Detailed Template excels in personalization and depth.
